In [1]:
import sk8s

In [2]:
import subprocess
import json
import time
from collections import Counter

In [3]:
sk8s.map(lambda a: str(a), [1,2,3])

['1', '2', '3']

In [13]:
def wait(jobs, delete=True):

    def get_job_status_json(jobs):
        stdout = subprocess.run(f"kubectl get jobs {' '.join(jobs)} -o json --ignore-not-found", 
                            check=True, shell=True,
                            stdout=subprocess.PIPE, stderr=subprocess.PIPE, encoding="utf-8").stdout
        try:
            return json.loads(stdout)
        except json.JSONDecodeError:
            return dict(kind="List", items=[])

    def check_job_status_json(data):
        assert(data["kind"] == "Job")
        if ("failed" in data["status"]) and (data["status"]["failed"] >= data["spec"]["backoffLimit"]):
            return "failed"
        if "succeeded" not in data["status"]:
            return "running"
        if data["status"]["succeeded"] == 1:
            return "success"

    def cleanup_job(job_name, delete=True):
        log_text = sk8s.logs(job_name, decode=True)
        if delete:
            sk8s.util.run_cmd(f"kubectl delete job {job_name}")
        assert(len(log_text.values()) == 1)
        return list(log_text.values())[0]

    if jobs.__class__ == str:
        jobs = [jobs]

    data = get_job_status_json(jobs)

    results = {d["metadata"]["name"]: check_job_status_json(d) for d in data["items"]}
    print(Counter(results.values()))

    actual_results = dict()  # nice variable name doofus 😜

    while True:
        data = get_job_status_json(jobs)
        results = {d["metadata"]["name"]: check_job_status_json(d) for d in data["items"]}
        #print(Counter(results.values()))
        print(results)
        if "running" not in results.values(): break

        for j, s in results.items():
            if s != "running":
                val = cleanup_job(j, delete=delete)
                actual_results[j] = val
                print(j, s, val, flush=True)

        time.sleep(1)

    # Now clean up whatever's left
    data = get_job_status_json(jobs)
    results = {d["metadata"]["name"]: check_job_status_json(d) for d in data["items"]}
    print(results)

    for j, s in results.items():
        val = cleanup_job(j, delete=delete)
        actual_results[j] = val
        print(j, s, val, flush=True)

    data = get_job_status_json(jobs)
    results = {d["metadata"]["name"]: check_job_status_json(d) for d in data["items"]}
    print(results)

    return [actual_results[job] for job in jobs]


from time import sleep
from numpy.random import rand as rand

n_jobs = 3
jobs = sk8s.map(lambda i: (sleep(i), i)[1], [rand() * 10 for i in range(n_jobs)], asynchro=True)
wait(jobs)

Counter({'running': 3})
{'job-uekyy': 'running', 'job-pbbxt': 'running', 'job-sfcjt': 'running'}
{'job-uekyy': 'running', 'job-pbbxt': 'running', 'job-sfcjt': 'running'}
{'job-uekyy': 'running', 'job-pbbxt': 'running', 'job-sfcjt': 'running'}
{'job-uekyy': 'running', 'job-pbbxt': 'running', 'job-sfcjt': 'running'}
{'job-uekyy': 'running', 'job-pbbxt': 'running', 'job-sfcjt': 'running'}
{'job-uekyy': 'running', 'job-pbbxt': 'success', 'job-sfcjt': 'running'}
job-pbbxt success 0.9314090911448614
{'job-uekyy': 'running', 'job-sfcjt': 'success'}
job-sfcjt success 2.4295227023123056
{'job-uekyy': 'running'}
{'job-uekyy': 'success'}
{'job-uekyy': 'success'}
job-uekyy success 7.985610689624255
{}


[7.985610689624255, 0.9314090911448614, 2.4295227023123056]